In [1]:
from path import path
import os
rootdir = path(u'/home/pfernique/Desktop/PyClangLite')
for filepath in (rootdir/'src'/'wrapper').walkfiles('*.[hc]*'):
    os.remove(filepath.abspath())

In [2]:
import subprocess
popen = subprocess.Popen(['clang++', '-x', 'c++', '-v', '-E', '/dev/null'], stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
out, err = popen.communicate()
sysincludes = err.splitlines()
headers = []
flags = ['-x', 'c++', '-std=c++11', '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS']
if '#include <...> search starts here:' not in sysincludes or 'End of search list.' not in sysincludes:
    warnings.warn('System includes not computed: parsing clang command output failed', Warning)
else:
    lindex = sysincludes.index('#include <...> search starts here:')
    rindex = sysincludes.index('End of search list.')
    sysincludes = sysincludes[lindex+1:rindex]
flags.extend(['-I'+str(path(sysinclude.strip()).abspath()) for sysinclude in sysincludes])
for sysinclude in sysincludes:
    sysinclude = path(sysinclude.strip())
    clanginclude = sysinclude/'clang'
    if clanginclude.exists():
        headers.extend([str(header.abspath()) for header in clanginclude.walkfiles('*.h')])
#selfcontained = rootdir/'src'/'wrapper'/'clang.h'
#selfcontained.write_lines(['#include <' + str(header) + '>' for header in headers])
#headers  = [selfcontained]
#headers = [str(header.abspath()) for header in headers]

In [3]:
headers

['/usr/local/include/clang/Lex/PreprocessingRecord.h',
 '/usr/local/include/clang/Lex/PTHManager.h',
 '/usr/local/include/clang/Lex/Pragma.h',
 '/usr/local/include/clang/Lex/ScratchBuffer.h',
 '/usr/local/include/clang/Lex/TokenLexer.h',
 '/usr/local/include/clang/Lex/Token.h',
 '/usr/local/include/clang/Lex/ModuleMap.h',
 '/usr/local/include/clang/Lex/Lexer.h',
 '/usr/local/include/clang/Lex/MultipleIncludeOpt.h',
 '/usr/local/include/clang/Lex/TokenConcatenation.h',
 '/usr/local/include/clang/Lex/PreprocessorLexer.h',
 '/usr/local/include/clang/Lex/ModuleLoader.h',
 '/usr/local/include/clang/Lex/PreprocessorOptions.h',
 '/usr/local/include/clang/Lex/HeaderSearch.h',
 '/usr/local/include/clang/Lex/PPCallbacks.h',
 '/usr/local/include/clang/Lex/ExternalPreprocessorSource.h',
 '/usr/local/include/clang/Lex/CodeCompletionHandler.h',
 '/usr/local/include/clang/Lex/HeaderMap.h',
 '/usr/local/include/clang/Lex/DirectoryLookup.h',
 '/usr/local/include/clang/Lex/PTHLexer.h',
 '/usr/local/incl

In [4]:
from autowig import autowig
import pickle
try:
    with open('pyclanglite.pkl', 'r') as f:
        asg = pickle.load(f)
except:
    asg = autowig.AbstractSemanticGraph()
    autowig.parser.plugin = 'pyclanglite'
    autowig.parser(asg, headers, flags=flags, bootstrap=False)
    with open('pyclanglite.pkl', 'w') as f:
        pickle.dump(asg, f)

In [5]:
from autowig.default_controller import refactoring
asg = refactoring(asg)
for node in asg.nodes('::clang::ASTUnit::top_level_(begin|end)'):
    if not node.return_type.boost_python_export:
        node.return_type.boost_python_export = True
for node in asg.nodes('^class ::std::(shared|weak|unique)_ptr$'):
    node.is_copyable = False
    node.is_smart_pointer = True
node = asg.nodes('::clang::HeaderSearch::NormalizeDashIncludePath').pop()
node.boost_python_export = False
for node in asg.nodes('::clang::DeclSpec::(TSCS|TSW|TSS|TST)_'):
    node.boost_python_export = False

In [6]:
subset = asg.nodes('::clang::tooling::buildASTFromCodeWithArgs')
classes = [asg['class ::clang::Type'], asg['class ::clang::Decl']]
subset += classes
subset += classes[0].subclasses(recursive=True)
subset += classes[1].subclasses(recursive=True)

In [7]:
from autowig.boost_python_generator import caching
caching(asg)

In [8]:
autowig.generator.plugin = 'boost_python'
wrappers = autowig.generator(asg, subset,
                             module=rootdir/'src'/'wrapper'/'_pyclanglite.cpp',
                             decorator=rootdir/'src'/'pyclanglite'/'_pyclanglite.py', closure=True)

In [9]:
for wrapper in wrappers:
    wrapper.write()

In [10]:
session = autowig.scons(rootdir, '-k', '-j3')
session

In [11]:
err = ''
feedback = autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4, indent=0,
                            force=True)
while not session.err == err:
    err = session.err
    exec feedback
    session = autowig.scons(rootdir,
                            '-k', '-j3')
    feedback = autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4, indent=0,
                            force=True)
session

In [10]:
%debug

ERROR: No traceback has been produced, nothing to debug.


In [11]:
feedback = autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4, indent=1,
                            force=True)
it = 0
while it < 10:
    controller += '\n' + feedback
    exec controller + "\n    return asg"
    autowig.controller['bootstrap'] = bootstrap_controller
    autowig.controller.plugin = 'bootstrap'
    asg = autowig.controller(asg)
    for wrapper in wrappers:
        if wrapper.on_disk:
            wrapper.write()
    session = autowig.scons(rootdir,
                            '-k', '-j6')
    feedback = autowig.feedback(session, rootdir, asg,
                                variantdir='build-scons',
                                tabsize=4, indent=1,
                                force=True)
    it = it + 1
session

In [12]:
%debug

ERROR: No traceback has been produced, nothing to debug.


In [13]:
[fct.prototype for fct in asg['class ::clang::DeclContext::decl_iterator'].functions()]

['bool  operator!=(class ::clang::DeclContext::decl_iterator , class ::clang::DeclContext::decl_iterator )',
 'bool  operator==(class ::clang::DeclContext::decl_iterator , class ::clang::DeclContext::decl_iterator )']

In [14]:
export = asg['class ::clang::ento::SVal'].boost_python_export

In [15]:
[dlc.__class__ for dlc in export.declarations]

[autowig.asg.ClassProxy]

In [16]:
sval = asg['class ::clang::ento::SVal']

In [17]:
sval.enumerations().pop().enumerators

[::clang::ento::SVal::BaseKind::UndefinedValKind,
 ::clang::ento::SVal::BaseKind::UnknownValKind,
 ::clang::ento::SVal::BaseKind::LocKind,
 ::clang::ento::SVal::BaseKind::NonLocKind]

In [18]:
import parse
wrappers = dict()
undefined = set()
for line in session.err.splitlines():
    parsed = parse.parse('build-scons/'+'{filename}:{row}:{column}:{message}', line)
    if parsed:
        row = int(parsed['row'])
        print row, parsed['filename']
        node = rootdir +'/' + parsed['filename']
        if node in asg:
            if not node in wrappers:
                wrappers[node] = [row]
            else:
                wrappers[node].append(row)

In [19]:
wrappers

{}